# Infer operator computation cost

This notebooks explores a way to predict the cost of operator Transpose based on some features.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

In [13]:
%load_ext mlprodict

## ONNX graph and measures

In [14]:
import numpy
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.algebra.onnx_ops import OnnxTranspose


def create_onnx_graph(perm=(0, 1, 2, 3), target_opset=14):
    tr = OnnxTranspose('X', perm=perm, output_names=['Y'], op_version=target_opset)
    return tr.to_onnx({'X': FloatTensorType([None, None, None, None])})


onx = create_onnx_graph()

%onnxview onx

In [18]:
from mlprodict.onnxrt import OnnxInference

onx = create_onnx_graph(perm=(1, 0, 3, 2))
oinf = OnnxInference(onx)
inputs = {'X': numpy.full((5, 6, 7, 8), 1, dtype=numpy.float32)}
res = oinf.run(inputs)['Y']
res.shape

(6, 5, 8, 7)

In [21]:
from onnxruntime import InferenceSession
sess = InferenceSession(onx.SerializeToString())
res = sess.run(None, inputs)[0]
res.shape

(6, 5, 8, 7)

In [51]:
from cpyquickhelper.numbers.speed_measure import measure_time

def measure_time_onnx(sess, X, number=50, repeat=30):
    inputs = {'X': X}
    return measure_time(lambda: sess.run(None, inputs), context=dict(sess=sess, inputs=inputs),
                        div_by_number=True, number=number, repeat=repeat)

X = numpy.random.random((3, 224, 224, 4)).astype(numpy.float32)
measure_time_onnx(sess, X)

{'average': 0.002489284733333079,
 'deviation': 0.0001954740394357934,
 'min_exec': 0.0022817859999940994,
 'max_exec': 0.002942614000003232,
 'repeat': 30,
 'number': 50,
 'context_size': 232}

## 4 dimensions, many permutation, one size

In [52]:
from itertools import permutations
from tqdm import tqdm
from pandas import DataFrame

shape = (13, 14, 15, 16)
X = numpy.random.random(shape).astype(numpy.float32)
obs = []
for perm in tqdm(permutations(list(range(len(X.shape))))):
    res = measure_time_onnx(sess, X)
    res['perm'] = perm
    res['shape'] = shape
    obs.append(res)

df = DataFrame(obs).sort_values('average')
df

24it [00:18,  1.27it/s]


,average,deviation,min_exec,max_exec,repeat,number,context_size,perm,shape
4,0.000424,0.000548,0.000142,0.002030,30,50,232,"(0, 3, 1, 2)","(13, 14, 15, 16)"
7,0.000451,0.000611,0.000143,0.001986,30,50,232,"(1, 0, 3, 2)","(13, 14, 15, 16)"
0,0.000455,0.000607,0.000139,0.002413,30,50,232,"(0, 1, 2, 3)","(13, 14, 15, 16)"
2,0.000484,0.000629,0.000141,0.002352,30,50,232,"(0, 2, 1, 3)","(13, 14, 15, 16)"
23,0.000495,0.000633,0.000142,0.002180,30,50,232,"(3, 2, 1, 0)","(13, 14, 15, 16)"
20,0.000505,0.000690,0.000140,0.002607,30,50,232,"(3, 1, 0, 2)","(13, 14, 15, 16)"
22,0.000522,0.000632,0.000141,0.002410,30,50,232,"(3, 2, 0, 1)","(13, 14, 15, 16)"
21,0.000523,0.000661,0.000141,0.002669,30,50,232,"(3, 1, 2, 0)","(13, 14, 15, 16)"
8,0.000529,0.000701,0.000142,0.002453,30,50,232,"(1, 2, 0, 3)","(13, 14, 15, 16)"
11,0.000531,0.000643,0.000141,0.002278,30,50,232,"(1, 3, 2, 0)","(13, 14, 15, 16)"


### features

* dernière dimension, taille
* nombre d'axes permutés